In [1]:
using Interpolations

In [2]:
include("Acrobots/src/Acrobots.jl")

Acrobots

In [3]:
using ProfileView
robot = Acrobots.acrobot()
xbar = Acrobots.AcrobotState{Float64}([pi; 0; 0; 0])
ybar = Acrobots.AcrobotOutput(xbar)
state = Acrobots.AcrobotState{Float64}([[pi; 0] + 0.2*(rand(2)-0.5); zeros(2)])
x0 = state
robot = Acrobots.acrobot()
input = Acrobots.AcrobotInput{Float64}(0)
dt = 0.001
vis = Acrobots.DrakeVisualizer(robot)
ts = 0:dt:5


controller_state = Acrobots.LQRState{Float64}(())
linear_sys = Acrobots.linearize(robot, 0, xbar, input)
controller = Acrobots.lqr(linear_sys, diagm([10,10,1,1]), eye(1))
linearizations = typeof(linear_sys)[]

Profile.clear()
elapsed = @elapsed @profile for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    input = Acrobots.output(controller, t, controller_state, output)
    statedot = Acrobots.dynamics(robot, t, state, input)
    state += statedot * dt
    push!(linearizations, Acrobots.linearize(robot, t, state, input))
    
    Acrobots.draw(vis, state)
end
@show elapsed / length(ts) * 1e6
# ProfileView.view()

(elapsed / length(ts)) * 1.0e6 = 629.6950179964007

629.6950179964007

In [4]:
lin = interpolate((ts,), linearizations, Gridded(Linear()));

In [5]:
controllers = Acrobots.tvlqr(lin, diagm([10.,10,1,1]), eye(1), diagm([10.,10,1,1]), eye(1));

In [13]:
state = x0 + 0.05 * Acrobots.AcrobotState(rand(4))
input = Acrobots.AcrobotInput{Float64}(0)

elapsed = @elapsed for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    input = Acrobots.output(controllers[t], t, controller_state, output)
    statedot = Acrobots.dynamics(robot, t, state, input)
    state += statedot * dt
    
    Acrobots.draw(vis, state)
end

1.942822757

In [7]:
robot = Acrobots.acrobot()
state = Acrobots.AcrobotState{Float64}([1.; 2.;3.; 4.])
input = Acrobots.AcrobotInput{Float64}(5.)
statedot = Acrobots.dynamics(robot, 0, state, input)
expected = Acrobots.AcrobotState{Float64}(3.0000, 4.0000, 21.3827, -18.4276)
@assert isapprox(statedot, expected, 1e-4)

linear_sys = Acrobots.linearize(robot, 0, state, input)
@assert isapprox(statedot, Acrobots.dynamics(linear_sys, 0, state, input), 1e-9)

vis = Acrobots.DrakeVisualizer(robot)
state = Acrobots.AcrobotState{Float64}([pi/4;pi/4;0;0])
Acrobots.draw(vis, state)